In [120]:
import requests
import pandas as pd

API_KEY = 'pk.eyJ1IjoicGV0ZXJuYWR5IiwiYSI6ImNsZW12a3JqbjAxOHc0NnBhZmFhZWliOWMifQ.yImyNototbp36myOh1hjvg'
url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
query = 'hospitals.json'
longitude = 29.941486
latitude = 31.219401
# radius = 5000
resloution = 0.005
params = {
    'access_token': API_KEY,
    'proximity': f'{longitude},{latitude}', # Specify latitude and longitude here
    'bbox': f'{longitude - resloution},{latitude - resloution},{longitude + resloution},{latitude + resloution}',
    'types': 'poi',
    'limit': 10
}

response = requests.get(url + query, params=params)
data = response.json()['features']

df = pd.DataFrame(data)
df

# the gio location retruend by the API is in the form of [longitude, latitude]
# where google maps API returns the location in the form of [latitude, longitude]

,id,type,place_type,relevance,properties,text,place_name,center,geometry,context
0,poi.111669233347,Feature,[poi],0.944444,"{'foursquare': '50082b0ce4b053787639d8ca', 'la...",Andalusia Hospital مستشفى السلامة,"Andalusia Hospital مستشفى السلامة, 35 Mohamed...","[29.942273, 31.216423]","{'coordinates': [29.942273, 31.216423], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."
1,poi.403726975003,Feature,[poi],0.944444,"{'foursquare': '4f3e0a8ae4b0eebf12c7e0cf', 'la...",ICC,"ICC, Smouha, Alexandria, Alexandria 21, Egypt","[29.942001, 31.216684]","{'coordinates': [29.942001, 31.216684], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."
2,poi.609885372475,Feature,[poi],0.944444,"{'foursquare': '4ecbc8bf722efb00f8ad76fd', 'la...",Sidi Gaber Military Hospital,"Sidi Gaber Military Hospital, Alexandria, Alex...","[29.941278, 31.224055]","{'coordinates': [29.941278, 31.224055], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."


In [26]:
import requests
import pandas as pd

# define the bounding box of Alexandria
bbox = "29.9503,31.1351,30.2153,31.3531"

# define the Overpass API query
query = f"""
    [out:json];
    node["amenity"="hospital"]({bbox});
    out;
"""

# send the request to the Overpass API and get the JSON data
response = requests.get(f"https://overpass-api.de/api/interpreter?data={query}")
data = response.json()

# extract the relevant data from the JSON and convert it to a DataFrame
hospitals = []
for element in data["elements"]:
    hospital = {}
    hospital["id"] = element["id"]
    hospital["name"] = element.get("tags", {}).get("name", "")
    hospital["lat"] = element["lat"]
    hospital["lon"] = element["lon"]
    hospitals.append(hospital)
df = pd.DataFrame(hospitals)
df

,id,name,lat,lon
0,366500797,معهد القلب,30.064775,31.209801
1,411676470,مستشفى القوات الجوية,30.062416,31.273825
2,411895460,مستشفي القاهرة التخصصى,30.094513,31.317617
3,414554419,دار المقطم,30.011238,31.285420
4,443368255,مركز الإيمان الطبي,30.064205,31.345582
...,...,...,...,...
81,9764636806,مستشفي جمال الدين الافغاني,30.096561,31.328417
82,9809753775,مستشفى بهيه لعلاج سرطان الثدي,30.004792,31.186687
83,9850700146,مستشفي سانت ريتا,30.094005,31.327129
84,9974961422,مستشفى الحلميه العسكرى,30.113921,31.318202


In [107]:
# using OpenStreetMap

import requests
import pandas as pd

# define the Overpass API query
query = """
    [out:json];
    node["amenity"="hospital"]({bbox});
    out;
"""

def get_hospitals_in_bbox(bbox):
    # send the request to the Overpass API and get the JSON data
    response = requests.get(f"https://overpass-api.de/api/interpreter?data={query.format(bbox=bbox)}")
    data = response.json()

    # extract the relevant data from the JSON and convert it to a DataFrame
    hospitals = []
    for element in data["elements"]:
        hospital = {}
        hospital["id"] = element["id"]
        hospital["name"] = element.get("tags", {}).get("name", "")
        hospital["lat"] = element["lat"]
        hospital["lon"] = element["lon"]
        hospitals.append(hospital)
    return pd.DataFrame(hospitals)

# define the bounding box of Alexandria
# alexandria_bbox = "29.9503,31.1351,30.2153,31.3531"
alexandria_bbox = "30.760719,28.975067,31.611288,30.791931"

# get hospitals in the initial bbox
df = get_hospitals_in_bbox(alexandria_bbox)
resloution = 0.5

# iterate over the rows of the DataFrame and get hospitals in each bbox
for i, row in df.iterrows():
    bbox = f"{row['lat']-resloution},{row['lon']-resloution},{row['lat']+resloution},{row['lon']+resloution}"
    hospitals_in_bbox = get_hospitals_in_bbox(bbox)
    df = pd.concat([df, hospitals_in_bbox])

# remove duplicates from the DataFrame
df.drop_duplicates(inplace=True)


In [121]:
df

,id,type,place_type,relevance,properties,text,place_name,center,geometry,context
0,poi.111669233347,Feature,[poi],0.944444,"{'foursquare': '50082b0ce4b053787639d8ca', 'la...",Andalusia Hospital مستشفى السلامة,"Andalusia Hospital مستشفى السلامة, 35 Mohamed...","[29.942273, 31.216423]","{'coordinates': [29.942273, 31.216423], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."
1,poi.403726975003,Feature,[poi],0.944444,"{'foursquare': '4f3e0a8ae4b0eebf12c7e0cf', 'la...",ICC,"ICC, Smouha, Alexandria, Alexandria 21, Egypt","[29.942001, 31.216684]","{'coordinates': [29.942001, 31.216684], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."
2,poi.609885372475,Feature,[poi],0.944444,"{'foursquare': '4ecbc8bf722efb00f8ad76fd', 'la...",Sidi Gaber Military Hospital,"Sidi Gaber Military Hospital, Alexandria, Alex...","[29.941278, 31.224055]","{'coordinates': [29.941278, 31.224055], 'type'...","[{'id': 'postcode.36419', 'mapbox_id': 'dXJuOm..."


In [116]:
df.to_csv('./hospitals in alex.csv')

In [53]:
df = pd.read_csv('./hospitals in alex.csv')

In [109]:
import random

NUM_OF_AFFILIATES = 25

affiliations = ['affiliate'] * NUM_OF_AFFILIATES + ['non affiliate'] * (len(df) - NUM_OF_AFFILIATES)

# randomly shuffle the affiliations list
random.shuffle(affiliations)

# create a new column in the dataframe with the affiliations
df['affiliation'] = affiliations

In [110]:
# assign a random number from 1 to 5 ambulance to each affiliate hospital

df['ambulances'] = None

# loop through each row in the dataframe
for index, row in df.iterrows():
    # check if the hospital is an 'affiliate'
    if row['affiliation'] == 'affiliate':
        # assign a random number of ambulances between 1 and 5
        num_ambulances = random.randint(1, 5)
        # update the 'ambulances' column for this row
        df.loc[index, 'ambulances'] = num_ambulances 

In [111]:
df['ambulances'] = df['ambulances'].apply(lambda x: 0 if x is None else x)

In [114]:
df = df[df['name'] != '']

In [115]:
df

,id,name,lat,lon,affiliation,ambulances
0,1179121562,مستشفى ماري مرقص,31.263625,29.991768,non affiliate,0
1,1698478723,Mostafa Kamel Hospital,31.225827,29.940808,affiliate,5
2,1844289361,معهد البحوث الطبيه,31.205290,29.927572,affiliate,5
3,1844290834,مستشفي المواساه,31.204391,29.928684,affiliate,2
4,3458193051,مستشفى شرق المدينة,31.262009,29.992335,non affiliate,2
5,3893917394,مستشفى إبراهيم عبيد التخصصي,31.237637,29.958088,non affiliate,0
6,3907737342,مستشفى الشاطبي,31.209381,29.911388,affiliate,1
7,3979548721,مستشفى هيفاء الطبى,31.096898,30.673614,non affiliate,0
8,4388651689,مستشفى مركز القلب الدولي icc,31.216874,29.942285,non affiliate,0
9,4563330892,مستشفى الانبا تكلا التخصصى,31.210927,29.926518,non affiliate,0
